In [52]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
# from sklearn.metrics import r2_score 

# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import KBinsDiscretizer

# from sklearn.pipeline import Pipeline,make_pipeline
# from sklearn.feature_selection import SelectKBest,chi2

from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LinearRegression

In [53]:
df = pd.read_csv('train.csv', usecols = ['Age','Fare', 'Survived'])

In [54]:
df

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500
...,...,...,...
886,0,27.0,13.0000
887,1,19.0,30.0000
888,0,NaN,23.4500
889,1,26.0,30.0000


In [55]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [56]:
df.dropna(inplace = True)

In [57]:
x = df.iloc[:,1:]
y = df.iloc[:,0]

In [58]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [59]:
clf = DecisionTreeClassifier()

In [60]:
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [61]:
accuracy_score(y_test,y_pred)

0.6363636363636364

In [62]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.6317097026604068

In [63]:
# Kmean Discretizer
kbin_age = KBinsDiscretizer(n_bins = 15,encode ='ordinal',strategy = 'quantile')
kbin_fare = KBinsDiscretizer(n_bins = 15,encode ='ordinal',strategy = 'quantile')

In [64]:
#pipeline of making columns [0],[1] are index
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [65]:
trf

ColumnTransformer(transformers=[('first',
                                 KBinsDiscretizer(encode='ordinal', n_bins=15),
                                 [0]),
                                ('second',
                                 KBinsDiscretizer(encode='ordinal', n_bins=15),
                                 [1])])

In [66]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)

In [67]:
# to check the information
trf.named_transformers_['first'].n_bins_

array([15])

In [68]:
# to check the information
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42,  6.  , 16.  , 19.  , 21.  , 23.  , 25.  , 28.  , 30.  ,
              32.  , 35.  , 38.  , 42.  , 47.  , 54.  , 80.  ])             ],
      dtype=object)

In [69]:
output = pd.DataFrame({
    'age':x_train['Age'],
    'age_trf':x_train_trf[:,0],
    'fare':x_train['Fare'],
    'fare_trf':x_train_trf[:,1],
})

In [70]:
output

,age,age_trf,fare,fare_trf
328,31.0,8.0,20.5250,8.0
73,26.0,6.0,14.4542,7.0
253,30.0,8.0,16.1000,7.0
719,33.0,9.0,7.7750,2.0
666,25.0,6.0,13.0000,6.0
...,...,...,...,...
92,46.0,12.0,61.1750,12.0
134,25.0,6.0,13.0000,6.0
337,41.0,11.0,134.5000,14.0
548,33.0,9.0,20.5250,8.0


In [71]:
clf = DecisionTreeClassifier()
clf.fit(x_train_trf,y_train)
y_pred2 = clf.predict(x_test_trf)

In [72]:
accuracy_score(y_test,y_pred2)

0.6363636363636364

In [73]:
x_trf = trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.6288732394366197